In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('./data/covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.3)

In [8]:
X_train

,age,gender,fever,cough,city
27,33,Female,102.0,Strong,Delhi
38,49,Female,101.0,Mild,Delhi
43,22,Female,99.0,Mild,Bangalore
69,73,Female,103.0,Mild,Delhi
22,71,Female,98.0,Strong,Kolkata
...,...,...,...,...,...
44,20,Male,102.0,Strong,Delhi
71,75,Female,104.0,Strong,Delhi
46,19,Female,101.0,Mild,Mumbai
73,34,Male,98.0,Strong,Kolkata


# Without Column Transformer

In [11]:
# Adding simple imputer
si = SimpleImputer()

X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(70, 1)

In [14]:
# Adding Odinal encoding
oe = OrdinalEncoder(categories=[['Mild','Strong']])

X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(70, 1)

In [16]:
# Adding One HotEncoding

ohe = OneHotEncoder(drop='first',sparse_output=False,dtype=np.int32)

X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_train[['gender','city']])

X_test_gender_city.shape

(70, 4)

In [17]:
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values
X_test_age = X_train.drop(columns=['gender','fever','cough','city']).values

X_test_age.shape

(70, 1)

In [19]:
X_train_tranformed = np.concatenate((X_train_age,X_train_gender_city,X_train_cough,X_train_fever),axis=1)
X_test_transformed = np.concatenate((X_train_age,X_train_gender_city,X_train_cough,X_train_fever),axis=1)

X_train_tranformed.shape

(70, 7)

In [21]:
X_train_tranformed

array([[ 33.      ,   0.      ,   1.      ,   0.      ,   0.      ,
          1.      , 102.      ],
       [ 49.      ,   0.      ,   1.      ,   0.      ,   0.      ,
          0.      , 101.      ],
       [ 22.      ,   0.      ,   0.      ,   0.      ,   0.      ,
          0.      ,  99.      ],
       [ 73.      ,   0.      ,   1.      ,   0.      ,   0.      ,
          0.      , 103.      ],
       [ 71.      ,   0.      ,   0.      ,   1.      ,   0.      ,
          1.      ,  98.      ],
       [  5.      ,   0.      ,   0.      ,   0.      ,   1.      ,
          1.      ,  98.      ],
       [ 71.      ,   1.      ,   0.      ,   1.      ,   0.      ,
          1.      , 100.921875],
       [ 20.      ,   0.      ,   0.      ,   0.      ,   1.      ,
          1.      , 100.921875],
       [ 19.      ,   1.      ,   1.      ,   0.      ,   0.      ,
          0.      , 101.      ],
       [ 83.      ,   1.      ,   0.      ,   1.      ,   0.      ,
          0.      , 103

# With Column Transformer

In [18]:
from sklearn.compose import ColumnTransformer

In [22]:
tranformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [23]:
tranformer.fit_transform(X_train).shape

(70, 7)

In [24]:
tranformer.fit_transform(X_test).shape

(30, 7)